# Mean Baseline

fork from https://www.kaggle.com/paulorzp/mean-baseline  
the difference is that I calculate mean value of `rv_lv_ratio_gte_1, rv_lv_ratio_lt_1, leftsided_pe, rightsided_pe, central_pe, chronic_pe, acute_and_chronic_pe` for positive exam only.

I modified so because 
`rv_lv_ratio_gte_1, rv_lv_ratio_lt_1, leftsided_pe, rightsided_pe, central_pe, chronic_pe, acute_and_chronic_pe` in train.csv is valid only when negative_exam_for_pe==0&&indeterminate==0 (I found above columns is all zero when negative_exam_for_pe==1||indeterminate==1)

**but above modification make the score worse**  
original (v3): 0.555  
fixed(v2): 0.657

I recheck the evaluation description and found that `rv_lv_ratio_gte_1, rv_lv_ratio_lt_1, leftsided_pe, rightsided_pe, central_pe, chronic_pe, acute_and_chronic_pe` is ALSO EVALUATED FOR ALL EXAMS INCLUDING NON-POSITIVE EXAMS. So this result is reasonable.

Though it may be still ok to eval `rv_lv_ratio_gte_1, rv_lv_ratio_lt_1` for all exam, 
It may be better to eval `leftsided_pe, rightsided_pe, central_pe, chronic_pe, acute_and_chronic_pe` for positive exam only because these columns are only valid for positive exams. Also, it is consistent with the fact that pe_present_on_image is evaluated for positive exam only.

It seems that scores for `leftsided_pe, rightsided_pe, central_pe, chronic_pe, acute_and_chronic_pe` is affected by exam level pos/neg prediction accuracy.

In [ ]:
USE_POS_MEAN = False

In [ ]:
import pandas as pd

In [ ]:
PATH = "../input/rsna-str-pulmonary-embolism-detection/"
train = pd.read_csv(PATH + "train.csv")
sub = pd.read_csv(PATH + "sample_submission.csv")

In [ ]:
feats = list(train.columns[3:5])+list(train.columns[8:12])+list(train.columns[13:17])
feats

In [ ]:
means = train[feats].mean().to_dict()
means

In [ ]:
means_pos_row = train.query("negative_exam_for_pe==0 and indeterminate==0")[feats].mean().to_dict()
means_pos_row

In [ ]:
# below values not used
# columns exept negative_exam_for_pe and indeterminate are all zero
means_not_pos_row = train.query("negative_exam_for_pe==1 or indeterminate==1")[feats].mean().to_dict()
means_not_pos_row

In [ ]:
POS_ONLY_ROWS = ["rv_lv_ratio_gte_1", "rv_lv_ratio_lt_1", "leftsided_pe", "rightsided_pe", "central_pe", "chronic_pe", "acute_and_chronic_pe"]

# pe_present_on_image prediction for sample_submission's id_only rows
sub['label'] = means['pe_present_on_image']

for feat in means.keys():
    sub.loc[sub.id.str.contains(feat, regex=False), 'label'] = means[feat]

if USE_POS_MEAN:
    for feat in POS_ONLY_ROWS:
        sub.loc[sub.id.str.contains(feat, regex=False), 'label'] = means_pos_row[feat]


In [ ]:
sub.to_csv('submission.csv', index = False, float_format='%.4g')